In [7]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [21]:
df = pd.read_csv("./preprocessed-data.csv")
df

,Date,Value
0,2010-01-01,388.91
1,2010-02-01,390.41
2,2010-03-01,391.37
3,2010-04-01,392.67
4,2010-05-01,393.21
...,...,...
179,2024-12-01,425.40
180,2025-01-01,426.65
181,2025-02-01,427.09
182,2025-03-01,428.15


In [23]:
def evaluateModel(df, predictedColumnName):
    validDf = df.dropna(subset=["Value", predictedColumnName])
    
    if len(validDf) == 0:
        return {
            "mse": np.nan,
            "rmse": np.nan,
            "mae": np.nan,
            "mape": np.nan
        }
    
    actual = validDf["Value"]
    predicted = validDf[predictedColumnName]
    
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(actual - predicted))
    
    nonZeroActual = actual[actual != 0]
    nonZeroPredicted = predicted[actual != 0]
    
    mape = np.mean(np.abs((nonZeroActual - nonZeroPredicted) / nonZeroActual)) * 100 if len(nonZeroActual) > 0 else np.nan
    
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "mape": mape
    }

In [25]:
def exponentialSmoothing(data, alpha):
    result = [data[0]]  # initial value
    for n in range(1, len(data)):
        result.append(alpha * data[n] + (1 - alpha) * result[n-1])
    return result

def predictWithExponentialSmoothing(data, alpha, forecast_horizon):
    smoothed = exponentialSmoothing(data, alpha)
    forecast_value = smoothed[-1]
    return [forecast_value] * forecast_horizon

In [27]:
def execExponentialSmoothing(alpha, forecast_horizon):
    values = df["Value"].values
    smoothed = exponentialSmoothing(values, alpha)

    df['ExpSmooth'] = smoothed

    lastDate = pd.to_datetime(df['Date'].iloc[-1])
    predictions = predictWithExponentialSmoothing(values, alpha, forecast_horizon)

    futureDates = []
    futureValues = []
    futureExpSmooths = []

    for i in range(forecast_horizon):
        newDate = lastDate + relativedelta(months=i+1)
        futureDates.append(newDate.strftime('%Y-%m-%d'))
        futureValues.append(np.nan)
        futureExpSmooths.append(predictions[i])

    futureDf = pd.DataFrame({
        'Date': futureDates,
        'Value': futureValues,
        'ExpSmooth': futureExpSmooths
    })

    extendedDf = pd.concat([df, futureDf], ignore_index=True)

    print(evaluateModel(df, "ExpSmooth"), "\n")
    print(extendedDf.to_string())
    return extendedDf

In [29]:
df = execExponentialSmoothing(alpha=0.3, forecast_horizon=12)
df

{'mse': 2.6153383702304565, 'rmse': 1.6172007822872385, 'mae': 1.4726006242587604, 'mape': 0.3616219155351945} 

           Date   Value   ExpSmooth
0    2010-01-01  388.91  388.910000
1    2010-02-01  390.41  389.360000
2    2010-03-01  391.37  389.963000
3    2010-04-01  392.67  390.775100
4    2010-05-01  393.21  391.505570
5    2010-06-01  392.38  391.767899
6    2010-07-01  390.41  391.360529
7    2010-08-01  388.54  390.514371
8    2010-09-01  387.03  389.469059
9    2010-10-01  387.43  388.857342
10   2010-11-01  388.87  388.861139
11   2010-12-01  389.99  389.199797
12   2011-01-01  391.50  389.889858
13   2011-02-01  392.05  390.537901
14   2011-03-01  392.80  391.216530
15   2011-04-01  393.44  391.883571
16   2011-05-01  394.41  392.641500
17   2011-06-01  393.95  393.034050
18   2011-07-01  392.72  392.939835
19   2011-08-01  390.33  392.156884
20   2011-09-01  389.28  391.293819
21   2011-10-01  389.19  390.662673
22   2011-11-01  390.48  390.607871
23   2011-12-01  392.06

,Date,Value,ExpSmooth
0,2010-01-01,388.91,388.91000
1,2010-02-01,390.41,389.36000
2,2010-03-01,391.37,389.96300
3,2010-04-01,392.67,390.77510
4,2010-05-01,393.21,391.50557
...,...,...,...
191,2025-12-01,NaN,427.32376
192,2026-01-01,NaN,427.32376
193,2026-02-01,NaN,427.32376
194,2026-03-01,NaN,427.32376


In [31]:
# Export it
outPath = "D:/Coding College/statistika-final-proj-main/exponential-smoothing-data.csv"
df.to_csv(outPath, index=False)

df

,Date,Value,ExpSmooth
0,2010-01-01,388.91,388.91000
1,2010-02-01,390.41,389.36000
2,2010-03-01,391.37,389.96300
3,2010-04-01,392.67,390.77510
4,2010-05-01,393.21,391.50557
...,...,...,...
191,2025-12-01,NaN,427.32376
192,2026-01-01,NaN,427.32376
193,2026-02-01,NaN,427.32376
194,2026-03-01,NaN,427.32376
